In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures, RobustScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping

# Load data
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df1_encoded = pd.get_dummies(df)
df2_encoded = pd.get_dummies(df_test)

X = df1_encoded.drop(columns=['price_doc'], axis=1)
y = df1_encoded['price_doc']

# Feature scaling
scaler = RobustScaler()
X = scaler.fit_transform(X)
df_test = scaler.transform(df_test)

# Dimensionality reduction with PCA
pca = PCA(n_components=15)
X = pca.fit_transform(X)
df_test = pca.transform(df_test)

# Polynomial features
poly = PolynomialFeatures(degree=3)
X = poly.fit_transform(X)
df_test = poly.transform(df_test)

# Model
n_features = X.shape[1]
model = Sequential()
model.add(Dense(150, input_dim=n_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(35, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=Adamax(learning_rate=0.01), loss='mean_squared_error')

# Model training
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, epochs=100, batch_size=256, callbacks=[early_stopping])

# Prediction
y_pred = model.predict(df_test)

# Save predictions to a CSV file
result_df = pd.DataFrame({'row ID': range(1, len(y_pred) + 1), 'price_doc': y_pred.flatten()})
result_df.to_csv('neural_29nov_4_improved.csv', index=False)


MemoryError: Unable to allocate 2.99 GiB for an array with shape (2214, 181507) and data type float64